<a href="https://colab.research.google.com/github/shinchan75034/TFE_Dataset_Dataframe_Conversion/blob/master/TFE_Dataset_Dataframe_Conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data structures and conversion

Structured data with fixed schema, such as a table or pandas dataframe, often appears as input data for TensorFlow workflow. In handling efficient training data streaming, TensorFlow leverages dataset structure quite often. Therefore it is important to understand how to convert between these data structures. Typically, pandas dataframe is converted to dataset for streaming during training, and dataset is converted to pandas for readability and verification of its content. This notebook demonstrates how conversion can be done at either direction. </br>

We will use Google's public dataset in BigQuery for demonstrating how to convert TensorFlow dataset to pandas dataframe, and for simplicity, we will convert Iris pandas dataframe to TensorFlow dataset.

In [95]:
 !pip install tensorflow-io

In [96]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [97]:
PROJECT_ID = "project1-190517" #@param {type:"string"}
! gcloud config set project $PROJECT_ID
%env GCLOUD_PROJECT=$PROJECT_ID

Updated property [core/project].
env: GCLOUD_PROJECT=project1-190517


In [98]:
!ls -lrt

total 12
drwxr-xr-x 1 root root 4096 May 29 18:19 sample_data
-rw-r--r-- 1 root root 2664 Jun  7 20:35 adc.json
-rw-r--r-- 1 root root 2285 Jun  7 20:37 project1-190517-858599adc951.json
lrwxrwxrwx 1 root root   33 Jun  7 20:37 service_account.json -> project1-190517-858599adc951.json


In [99]:
from google.colab import files
uploaded = files.upload()

Saving project1-190517-858599adc951.json to project1-190517-858599adc951 (1).json


In [0]:
!rm service_account.json
!ln -s project1-190517-858599adc951.json service_account.json

In [101]:
!ls -lrt ./service_account.json

lrwxrwxrwx 1 root root 33 Jun  8 01:23 ./service_account.json -> project1-190517-858599adc951.json


In [0]:
!cat ./service_account.json

In [0]:
from google.cloud import bigquery
from google.oauth2 import service_account

# TODO(developer): Set key_path to the path to the service account key
#                  file.
key_path = "./service_account.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(
    credentials=credentials,
    project=credentials.project_id,
)

In [104]:
client

In [0]:
project_id ="project1-190517"

In [0]:
import tensorflow as tf
from tensorflow_io.bigquery import BigQueryClient

In [0]:
import pandas as pd

In [0]:
PROJECT_ID = "project1-190517" # This is from what you created in your Google Cloud Account. 
DATASET_GCP_PROJECT_ID = "bigquery-public-data"
DATASET_ID = "covid19_ecdc"
TABLE_ID = "covid_19_geographic_distribution_worldwide"

In [0]:
client = BigQueryClient()
read_session = client.read_session(
    "projects/" + PROJECT_ID,
    DATASET_GCP_PROJECT_ID, TABLE_ID, DATASET_ID,
    ["countries_and_territories",
     "geo_id",
     "country_territory_code",
     "year",
     "month",
     "day",
     "confirmed_cases",
     "daily_deaths",
     "pop_data_2018"
     ],
    [tf.string,
     tf.string,
     tf.string,
     tf.int64,
     tf.int64,
     tf.int64,
     tf.int64,
     tf.int64,
     tf.int64
     
     
     ],
      requested_streams=10
)

In [0]:
batch_size = 10
dataset = read_session.parallel_read_rows(sloppy=True).batch(batch_size)

In [128]:
type(dataset)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [0]:
itr = tf.compat.v1.data.make_one_shot_iterator(dataset)

In [111]:
next(itr)

OrderedDict([('confirmed_cases',
              <tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])>),
             ('countries_and_territories',
              <tf.Tensor: shape=(10,), dtype=string, numpy=
              array([b'United_Arab_Emirates', b'Afghanistan', b'Armenia', b'Austria',
                     b'Australia', b'Azerbaijan', b'Belgium', b'Bahrain', b'Brazil',
                     b'Belarus'], dtype=object)>),
             ('country_territory_code',
              <tf.Tensor: shape=(10,), dtype=string, numpy=
              array([b'ARE', b'AFG', b'ARM', b'AUT', b'AUS', b'AZE', b'BEL', b'BHR',
                     b'BRA', b'BLR'], dtype=object)>),
             ('daily_deaths',
              <tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])>),
             ('day',
              <tf.Tensor: shape=(10,), dtype=int64, numpy=array([31, 31, 31, 31, 31, 31, 31, 31, 31, 31])>),
             ('geo_id', <tf.Tensor: shape=(1

## From dataset to dataframe


This is often necessary in order to check the actual data values in the dataset. Converting a dataset to pandas dataframe makes it humanly readable helps ascertain the content of dataset. </br>
Inside the dataset is really OrderedDict data structure. So if we want to format it and output it in a humanly readable fashion such as a Pandas dataframe, we need to somehow convert an OrderedDict to dataframe. See [this reference](https://stackoverflow.com/questions/44365209/generate-a-pandas-dataframe-from-ordereddict "OrderedDict to dataframe")

In [190]:
for raw_record in dataset.take(1):
  df = pd.DataFrame(raw_record, columns = raw_record.keys())
  print(df)


   confirmed_cases countries_and_territories  ... pop_data_2018  year
0                0   b'United_Arab_Emirates'  ...       9630959  2019
1                0            b'Afghanistan'  ...      37172386  2019
2                0                b'Armenia'  ...       2951776  2019
3                0                b'Austria'  ...       8847037  2019
4                0              b'Australia'  ...      24992369  2019
5                0             b'Azerbaijan'  ...       9942334  2019
6                0                b'Belgium'  ...      11422068  2019
7                0                b'Bahrain'  ...       1569439  2019
8                0                 b'Brazil'  ...     209469333  2019
9                0                b'Belarus'  ...       9485386  2019

[10 rows x 9 columns]


In [191]:
df

,confirmed_cases,countries_and_territories,country_territory_code,daily_deaths,day,geo_id,month,pop_data_2018,year
0,0,b'United_Arab_Emirates',b'ARE',0,31,b'AE',12,9630959,2019
1,0,b'Afghanistan',b'AFG',0,31,b'AF',12,37172386,2019
2,0,b'Armenia',b'ARM',0,31,b'AM',12,2951776,2019
3,0,b'Austria',b'AUT',0,31,b'AT',12,8847037,2019
4,0,b'Australia',b'AUS',0,31,b'AU',12,24992369,2019
5,0,b'Azerbaijan',b'AZE',0,31,b'AZ',12,9942334,2019
6,0,b'Belgium',b'BEL',0,31,b'BE',12,11422068,2019
7,0,b'Bahrain',b'BHR',0,31,b'BH',12,1569439,2019
8,0,b'Brazil',b'BRA',0,31,b'BR',12,209469333,2019
9,0,b'Belarus',b'BLR',0,31,b'BY',12,9485386,2019


For readability, lets clean up the byte string in the three columns that are coded as string. This can be done by decoding these columns to `utf-8`. See [this reference](https://stackoverflow.com/questions/46696679/removing-b-from-string-column-in-a-pandas-dataframe "remove b' from string columns")

In [192]:
df.apply(lambda x: x.str.decode('utf-8') if x.name in ['countries_and_territories', 'country_territory_code', 'geo_id'] else x) 

,confirmed_cases,countries_and_territories,country_territory_code,daily_deaths,day,geo_id,month,pop_data_2018,year
0,0,United_Arab_Emirates,ARE,0,31,AE,12,9630959,2019
1,0,Afghanistan,AFG,0,31,AF,12,37172386,2019
2,0,Armenia,ARM,0,31,AM,12,2951776,2019
3,0,Austria,AUT,0,31,AT,12,8847037,2019
4,0,Australia,AUS,0,31,AU,12,24992369,2019
5,0,Azerbaijan,AZE,0,31,AZ,12,9942334,2019
6,0,Belgium,BEL,0,31,BE,12,11422068,2019
7,0,Bahrain,BHR,0,31,BH,12,1569439,2019
8,0,Brazil,BRA,0,31,BR,12,209469333,2019
9,0,Belarus,BLR,0,31,BY,12,9485386,2019


We have demonstrated how to convert a dataset into a Pandas dataframe.

## Conversion between pandas and tf.dataset

What we have so far is a pandas data frame from Iris dataset, and an example dataset that we read from Google public data using BigQuery. It turns out that these two data structures are very important and heavily used in many TensorFlow examples or use cases. Very often, it is necessary to know how to convert between these data structures in order to understand the examples and build model workflow. So lets spend some time to understand how to convert one structure to another. 

### Pandas to dataset
Lets convert Iris training dataframe to a dataset. We will keep features and target together. 

[Reference](https://medium.com/when-i-work-data/converting-a-pandas-dataframe-into-a-tensorflow-dataset-752f3783c168 "pandas to dataset")

In [0]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

In [115]:
type(train)

pandas.core.frame.DataFrame

In [118]:
# The label column has now been removed from the features.
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [0]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [0]:
column_names_idx = train.keys()

Lets separate target from features.

In [0]:
feature_list = column_names_idx.drop('Species')
target = column_names_idx.drop(feature_list)

In [121]:
print(feature_list, ', ', target)

Index(['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth'], dtype='object') ,  Index(['Species'], dtype='object')


In [0]:
training_dataset = tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(train[feature_list].values, tf.float32),
            tf.cast(train[target].values, tf.int32)
        )
    )

In [127]:
n = 0
for features_tensor, target_tensor in training_dataset:
  if n < 5:
    print(f'features:{features_tensor} target:{target_tensor}')
    n +=1

features:[6.4 2.8 5.6 2.2] target:[2]
features:[5.  2.3 3.3 1. ] target:[1]
features:[4.9 2.5 4.5 1.7] target:[2]
features:[4.9 3.1 1.5 0.1] target:[0]
features:[5.7 3.8 1.7 0.3] target:[0]


Now we have converted Pandas dataframe to a tf.dataset.